# Advanced VQE: Parameter Studies and Optimization

This notebook explores advanced VQE features including:
- Ansatz depth studies
- Optimizer comparisons
- Scaling analysis
- Hamiltonian weight tuning

**Author:** Tommaso Marena (marena@cua.edu)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from pathlib import Path
import sys

sys.path.insert(0, str(Path.cwd().parent / 'src'))

from quantum_protein_folding.models import VQEFoldingModel
from quantum_protein_folding.analysis import (    plot_convergence, plot_conformation_2d, compute_rmsd, analyze_convergence
)
from quantum_protein_folding.quantum.hamiltonian import compute_exact_ground_state

## 1. Ansatz Depth Study

How does ansatz depth affect solution quality?

In [ ]:
sequence = "HPHPHPHH"  # 8-residue sequence
depths = [1, 2, 3, 4]

results = {}

for depth in depths:
    print(f"\nRunning VQE with depth={depth}...")
    
    model = VQEFoldingModel(
        sequence=sequence,
        lattice_dim=2,
        ansatz_depth=depth,
        optimizer='COBYLA',
        shots=1024
    )
    
    start_time = time.time()
    result = model.run(maxiter=100)
    elapsed = time.time() - start_time
    
    results[depth] = {
        'energy': result.optimal_value,
        'time': elapsed,
        'n_params': model.solver.n_params,
        'convergence': result.convergence_history
    }
    
    print(f"  Energy: {result.optimal_value:.4f}")
    print(f"  Time: {elapsed:.2f}s")
    print(f"  Parameters: {model.solver.n_params}")

In [ ]:
# Plot depth comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Energy vs depth
ax1 = axes[0]
energies = [results[d]['energy'] for d in depths]
ax1.plot(depths, energies, 'o-', linewidth=2, markersize=10)
ax1.set_xlabel('Ansatz Depth', fontsize=12)
ax1.set_ylabel('Final Energy', fontsize=12)
ax1.set_title('Energy vs Ansatz Depth', fontweight='bold')
ax1.grid(True, alpha=0.3)

# Time vs depth
ax2 = axes[1]
times = [results[d]['time'] for d in depths]
ax2.plot(depths, times, 's-', linewidth=2, markersize=10, color='orange')
ax2.set_xlabel('Ansatz Depth', fontsize=12)
ax2.set_ylabel('Time (s)', fontsize=12)
ax2.set_title('Time vs Ansatz Depth', fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Optimizer Comparison

Compare different classical optimizers.

In [ ]:
optimizers = ['COBYLA', 'SLSQP', 'SPSA']
opt_results = {}

for opt in optimizers:
    print(f"\nTesting optimizer: {opt}")
    
    model = VQEFoldingModel(
        sequence=sequence,
        lattice_dim=2,
        ansatz_depth=2,
        optimizer=opt,
        shots=1024
    )
    
    start = time.time()
    result = model.run(maxiter=50)
    elapsed = time.time() - start
    
    opt_results[opt] = {
        'energy': result.optimal_value,
        'time': elapsed,
        'iterations': result.n_iterations,
        'convergence': result.convergence_history
    }
    
    print(f"  Final energy: {result.optimal_value:.4f}")
    print(f"  Time: {elapsed:.2f}s")

In [ ]:
# Plot convergence comparison
plt.figure(figsize=(12, 6))

for opt in optimizers:
    history = opt_results[opt]['convergence']
    plt.plot(history, label=f"{opt} (E={opt_results[opt]['energy']:.3f})", linewidth=2)

plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Energy', fontsize=12)
plt.title('Optimizer Comparison', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Hamiltonian Weight Tuning

Study the effect of constraint and bias weights.

In [ ]:
constraint_weights = [1.0, 5.0, 10.0, 20.0]
weight_results = {}

for cw in constraint_weights:
    print(f"\nConstraint weight: {cw}")
    
    # Create new encoding with this weight
    from quantum_protein_folding.data.loaders import load_hp_sequence
    from quantum_protein_folding.data.preprocess import map_to_lattice
    from quantum_protein_folding.quantum.vqe import VQESolver
    
    seq = load_hp_sequence(sequence)
    encoding = map_to_lattice(
        seq,
        lattice_dim=2,
        constraint_weight=cw,
        bias_weight=0.1
    )
    
    solver = VQESolver(
        hamiltonian=encoding.hamiltonian,
        n_qubits=encoding.n_qubits,
        ansatz_depth=2,
        optimizer='COBYLA'
    )
    
    result = solver.run(maxiter=50)
    
    weight_results[cw] = {
        'energy': result.optimal_value,
        'convergence': result.convergence_history
    }
    
    print(f"  Energy: {result.optimal_value:.4f}")

In [ ]:
# Plot weight effect
plt.figure(figsize=(10, 6))

weights = list(weight_results.keys())
energies = [weight_results[w]['energy'] for w in weights]

plt.plot(weights, energies, 'o-', linewidth=2, markersize=10)
plt.xlabel('Constraint Weight (λ)', fontsize=12)
plt.ylabel('Final Energy', fontsize=12)
plt.title('Effect of Constraint Weight on Solution Quality', fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Chain Length Scaling

Analyze resource scaling with protein length.

In [ ]:
sequences = {
    6: "HPHHPH",
    8: "HPHPHPHH",
    10: "HPHPPHHPHH",
    12: "HPHPPHHPHHPH"
}

scaling_results = {}

for length, seq in sequences.items():
    print(f"\nChain length: {length}")
    
    model = VQEFoldingModel(
        sequence=seq,
        lattice_dim=2,
        ansatz_depth=2,
        optimizer='COBYLA'
    )
    
    start = time.time()
    result = model.run(maxiter=50)
    elapsed = time.time() - start
    
    scaling_results[length] = {
        'n_qubits': model.encoding.n_qubits,
        'n_params': model.solver.n_params,
        'time': elapsed,
        'energy': result.optimal_value
    }
    
    print(f"  Qubits: {model.encoding.n_qubits}")
    print(f"  Parameters: {model.solver.n_params}")
    print(f"  Time: {elapsed:.2f}s")

In [ ]:
# Plot scaling analysis
from quantum_protein_folding.analysis.plots import plot_scaling_analysis

plot_scaling_analysis(scaling_results)

## Summary

Key findings:
1. Deeper ansatzes improve solution quality but increase runtime
2. COBYLA generally performs well for VQE protein folding
3. Constraint weights must be tuned to balance objectives
4. Qubit count scales approximately linearly with chain length

These insights guide practical quantum algorithm design for protein folding.